In [1]:
import os
import yfinance as yf
from datetime import datetime
import pandas as pd
import pytz
import logging

import matplotlib.pyplot as plt

In [24]:
import numpy as np

In [174]:
msft = yf.Ticker('MSFT')
START = datetime(1980,1,1)
END = datetime(2020,4,24)
hist = msft.history(start = START, end = END)

In [205]:
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1986-03-13,0.06,0.07,0.06,0.06,1031788800,0.0,0.0
1986-03-14,0.06,0.07,0.06,0.06,308160000,0.0,0.0
1986-03-17,0.06,0.07,0.06,0.07,133171200,0.0,0.0
1986-03-18,0.07,0.07,0.06,0.06,67766400,0.0,0.0
1986-03-19,0.06,0.06,0.06,0.06,47894400,0.0,0.0
...,...,...,...,...,...,...,...
2020-04-17,179.50,180.00,175.87,178.60,52765600,0.0,0.0
2020-04-20,176.63,178.75,174.99,175.06,36669600,0.0,0.0
2020-04-21,173.50,173.67,166.11,167.82,56203700,0.0,0.0


In [210]:
def dta_to_candlestick(data):
    l = len(data)
    # Make candlestick picture
    layout = go.Layout(xaxis=dict(ticks='',
                                  showgrid=False,
                                  showticklabels=False,
                                  rangeslider=dict (visible = False)),
                       yaxis=dict(ticks='',
                                  showgrid=False,
                                  showticklabels=False),
                       paper_bgcolor='rgba(0,0,0,0)',
                       plot_bgcolor='rgba(0,0,0,0)')
    fig = go.Figure(data=[go.Candlestick(x=np.linspace(1,l,l),
                                    open=data.Open,
                                    high=data.High,
                                    low=data.Low,
                                    close=data.Close)],
                   layout=layout)
    fig.write_image("images/fig-{}.png".format(data.Open[0]))
    
    # Convert to numpy array
    im = Image.open('images/fig-{}.png'.format(data.Open[0]))
    data = np.asarray(im)
    
    # Return the first channel of the image
    return data[:,:,0]

In [211]:
def dta_transformation(data, est_h):
    # Make sure data has sufficient columns
    assert 'Open' in data.columns
    assert 'High' in data.columns
    assert 'Low' in data.columns
    assert 'Close' in data.columns
    
    data['lag_close'] = data['Close'].shift(1)
    data['Indicator'] = np.where(data['Close'] > data['lag_close'], 1, 0)
    
    x = []
    y = []
    for i in range(est_h, data.shape[0]):
        sub_dta = data.iloc[i-est_h:i]
        
        y_i = data.iloc[i]['Indicator']
        x_i = dta_to_candlestick(sub_dta)
        
        y.append(y_i)
        x.append(x_i)
        
        print("{}/{}".format(i-est_h, data.shape[0]-est_h))
    
    return x, y

In [212]:
x, y = dta_transformation(hist, 30)

0/8569
1/8569
2/8569
3/8569
4/8569
5/8569
6/8569
7/8569
8/8569
9/8569
10/8569
11/8569
12/8569
13/8569
14/8569
15/8569
16/8569
17/8569
18/8569
19/8569
20/8569
21/8569
22/8569
23/8569
24/8569
25/8569
26/8569
27/8569
28/8569
29/8569
30/8569
31/8569
32/8569
33/8569
34/8569
35/8569
36/8569
37/8569
38/8569
39/8569
40/8569
41/8569
42/8569
43/8569
44/8569
45/8569
46/8569
47/8569
48/8569
49/8569
50/8569
51/8569
52/8569
53/8569
54/8569
55/8569
56/8569
57/8569
58/8569
59/8569
60/8569
61/8569
62/8569
63/8569
64/8569
65/8569
66/8569
67/8569
68/8569
69/8569
70/8569
71/8569
72/8569
73/8569
74/8569
75/8569
76/8569
77/8569
78/8569
79/8569
80/8569
81/8569
82/8569
83/8569
84/8569
85/8569
86/8569
87/8569
88/8569
89/8569
90/8569
91/8569
92/8569
93/8569
94/8569
95/8569
96/8569
97/8569
98/8569
99/8569
100/8569
101/8569
102/8569
103/8569
104/8569
105/8569
106/8569
107/8569
108/8569
109/8569
110/8569
111/8569
112/8569
113/8569
114/8569
115/8569
116/8569
117/8569
118/8569
119/8569
120/8569
121/8569
122/8569
123

923/8569
924/8569
925/8569
926/8569
927/8569
928/8569
929/8569
930/8569
931/8569
932/8569
933/8569
934/8569
935/8569
936/8569
937/8569
938/8569
939/8569
940/8569
941/8569
942/8569
943/8569
944/8569
945/8569
946/8569
947/8569
948/8569
949/8569
950/8569
951/8569
952/8569
953/8569
954/8569
955/8569
956/8569
957/8569
958/8569
959/8569
960/8569
961/8569
962/8569
963/8569
964/8569
965/8569
966/8569
967/8569
968/8569
969/8569
970/8569
971/8569
972/8569
973/8569
974/8569
975/8569
976/8569
977/8569
978/8569
979/8569
980/8569
981/8569
982/8569
983/8569
984/8569
985/8569
986/8569
987/8569
988/8569
989/8569
990/8569
991/8569
992/8569
993/8569
994/8569
995/8569
996/8569
997/8569
998/8569
999/8569
1000/8569
1001/8569
1002/8569
1003/8569
1004/8569
1005/8569
1006/8569
1007/8569
1008/8569
1009/8569
1010/8569
1011/8569
1012/8569
1013/8569
1014/8569
1015/8569
1016/8569
1017/8569
1018/8569
1019/8569
1020/8569
1021/8569
1022/8569
1023/8569
1024/8569
1025/8569
1026/8569
1027/8569
1028/8569
1029/8569
1030/85

1751/8569
1752/8569
1753/8569
1754/8569
1755/8569
1756/8569
1757/8569
1758/8569
1759/8569
1760/8569
1761/8569
1762/8569
1763/8569
1764/8569
1765/8569
1766/8569
1767/8569
1768/8569
1769/8569
1770/8569
1771/8569
1772/8569
1773/8569
1774/8569
1775/8569
1776/8569
1777/8569
1778/8569
1779/8569
1780/8569
1781/8569
1782/8569
1783/8569
1784/8569
1785/8569
1786/8569
1787/8569
1788/8569
1789/8569
1790/8569
1791/8569
1792/8569
1793/8569
1794/8569
1795/8569
1796/8569
1797/8569
1798/8569
1799/8569
1800/8569
1801/8569
1802/8569
1803/8569
1804/8569
1805/8569
1806/8569
1807/8569
1808/8569
1809/8569
1810/8569
1811/8569
1812/8569
1813/8569
1814/8569
1815/8569
1816/8569
1817/8569
1818/8569
1819/8569
1820/8569
1821/8569
1822/8569
1823/8569
1824/8569
1825/8569
1826/8569
1827/8569
1828/8569
1829/8569
1830/8569
1831/8569
1832/8569
1833/8569
1834/8569
1835/8569
1836/8569
1837/8569
1838/8569
1839/8569
1840/8569
1841/8569
1842/8569
1843/8569
1844/8569
1845/8569
1846/8569
1847/8569
1848/8569
1849/8569
1850/8569


2572/8569
2573/8569
2574/8569
2575/8569
2576/8569
2577/8569
2578/8569
2579/8569
2580/8569
2581/8569
2582/8569
2583/8569
2584/8569
2585/8569
2586/8569
2587/8569
2588/8569
2589/8569
2590/8569
2591/8569
2592/8569
2593/8569
2594/8569
2595/8569
2596/8569
2597/8569
2598/8569
2599/8569
2600/8569
2601/8569
2602/8569
2603/8569
2604/8569
2605/8569
2606/8569
2607/8569
2608/8569
2609/8569
2610/8569
2611/8569
2612/8569
2613/8569
2614/8569
2615/8569
2616/8569
2617/8569
2618/8569
2619/8569
2620/8569
2621/8569
2622/8569
2623/8569
2624/8569
2625/8569
2626/8569
2627/8569
2628/8569
2629/8569
2630/8569
2631/8569
2632/8569
2633/8569
2634/8569
2635/8569
2636/8569
2637/8569
2638/8569
2639/8569
2640/8569
2641/8569
2642/8569
2643/8569
2644/8569
2645/8569
2646/8569
2647/8569
2648/8569
2649/8569
2650/8569
2651/8569
2652/8569
2653/8569
2654/8569
2655/8569
2656/8569
2657/8569
2658/8569
2659/8569
2660/8569
2661/8569
2662/8569
2663/8569
2664/8569
2665/8569
2666/8569
2667/8569
2668/8569
2669/8569
2670/8569
2671/8569


3393/8569
3394/8569
3395/8569
3396/8569
3397/8569
3398/8569
3399/8569
3400/8569
3401/8569
3402/8569
3403/8569
3404/8569
3405/8569
3406/8569
3407/8569
3408/8569
3409/8569
3410/8569
3411/8569
3412/8569
3413/8569
3414/8569
3415/8569
3416/8569
3417/8569
3418/8569
3419/8569
3420/8569
3421/8569
3422/8569
3423/8569
3424/8569
3425/8569
3426/8569
3427/8569
3428/8569
3429/8569
3430/8569
3431/8569
3432/8569
3433/8569
3434/8569
3435/8569
3436/8569
3437/8569
3438/8569
3439/8569
3440/8569
3441/8569
3442/8569
3443/8569
3444/8569
3445/8569
3446/8569
3447/8569
3448/8569
3449/8569
3450/8569
3451/8569
3452/8569
3453/8569
3454/8569
3455/8569
3456/8569
3457/8569
3458/8569
3459/8569
3460/8569
3461/8569
3462/8569
3463/8569
3464/8569
3465/8569
3466/8569
3467/8569
3468/8569
3469/8569
3470/8569
3471/8569
3472/8569
3473/8569
3474/8569
3475/8569
3476/8569
3477/8569
3478/8569
3479/8569
3480/8569
3481/8569
3482/8569
3483/8569
3484/8569
3485/8569
3486/8569
3487/8569
3488/8569
3489/8569
3490/8569
3491/8569
3492/8569


...

5033/8569
5034/8569
5035/8569
5036/8569
5037/8569
5038/8569
5039/8569
5040/8569
5041/8569
5042/8569
5043/8569
5044/8569
5045/8569
5046/8569
5047/8569
5048/8569
5049/8569
5050/8569
5051/8569
5052/8569
5053/8569
5054/8569
5055/8569
5056/8569
5057/8569
5058/8569
5059/8569
5060/8569
5061/8569
5062/8569
5063/8569
5064/8569
5065/8569
5066/8569
5067/8569
5068/8569
5069/8569
5070/8569
5071/8569
5072/8569
5073/8569
5074/8569
5075/8569
5076/8569
5077/8569
5078/8569
5079/8569
5080/8569
5081/8569
5082/8569
5083/8569
5084/8569
5085/8569
5086/8569
5087/8569
5088/8569
5089/8569
5090/8569
5091/8569
5092/8569
5093/8569
5094/8569
5095/8569
5096/8569
5097/8569
5098/8569
5099/8569
5100/8569
5101/8569
5102/8569
5103/8569
5104/8569
5105/8569
5106/8569
5107/8569
5108/8569
5109/8569
5110/8569
5111/8569
5112/8569
5113/8569
5114/8569
5115/8569
5116/8569
5117/8569
5118/8569
5119/8569
5120/8569
5121/8569
5122/8569
5123/8569
5124/8569
5125/8569
5126/8569
5127/8569
5128/8569
5129/8569
5130/8569
5131/8569
5132/8569


5854/8569
5855/8569
5856/8569
5857/8569
5858/8569
5859/8569
5860/8569
5861/8569
5862/8569
5863/8569
5864/8569
5865/8569
5866/8569
5867/8569
5868/8569
5869/8569
5870/8569
5871/8569
5872/8569
5873/8569
5874/8569
5875/8569
5876/8569
5877/8569
5878/8569
5879/8569
5880/8569
5881/8569
5882/8569
5883/8569
5884/8569
5885/8569
5886/8569
5887/8569
5888/8569
5889/8569
5890/8569
5891/8569
5892/8569
5893/8569
5894/8569
5895/8569
5896/8569
5897/8569
5898/8569
5899/8569
5900/8569
5901/8569
5902/8569
5903/8569
5904/8569
5905/8569
5906/8569
5907/8569
5908/8569
5909/8569
5910/8569
5911/8569
5912/8569
5913/8569
5914/8569
5915/8569
5916/8569
5917/8569
5918/8569
5919/8569
5920/8569
5921/8569
5922/8569
5923/8569
5924/8569
5925/8569
5926/8569
5927/8569
5928/8569
5929/8569
5930/8569
5931/8569
5932/8569
5933/8569
5934/8569
5935/8569
5936/8569
5937/8569
5938/8569
5939/8569
5940/8569
5941/8569
5942/8569
5943/8569
5944/8569
5945/8569
5946/8569
5947/8569
5948/8569
5949/8569
5950/8569
5951/8569
5952/8569
5953/8569


6674/8569
6675/8569
6676/8569
6677/8569
6678/8569
6679/8569
6680/8569
6681/8569
6682/8569
6683/8569
6684/8569
6685/8569
6686/8569
6687/8569
6688/8569
6689/8569
6690/8569
6691/8569
6692/8569
6693/8569
6694/8569
6695/8569
6696/8569
6697/8569
6698/8569
6699/8569
6700/8569
6701/8569
6702/8569
6703/8569
6704/8569
6705/8569
6706/8569
6707/8569
6708/8569
6709/8569
6710/8569
6711/8569
6712/8569
6713/8569
6714/8569
6715/8569
6716/8569
6717/8569
6718/8569
6719/8569
6720/8569
6721/8569
6722/8569
6723/8569
6724/8569
6725/8569
6726/8569
6727/8569
6728/8569
6729/8569
6730/8569
6731/8569
6732/8569
6733/8569
6734/8569
6735/8569
6736/8569
6737/8569
6738/8569
6739/8569
6740/8569
6741/8569
6742/8569
6743/8569
6744/8569
6745/8569
6746/8569
6747/8569
6748/8569
6749/8569
6750/8569
6751/8569
6752/8569
6753/8569
6754/8569
6755/8569
6756/8569
6757/8569
6758/8569
6759/8569
6760/8569
6761/8569
6762/8569
6763/8569
6764/8569
6765/8569
6766/8569
6767/8569
6768/8569
6769/8569
6770/8569
6771/8569
6772/8569
6773/8569


7494/8569
7495/8569
7496/8569
7497/8569
7498/8569
7499/8569
7500/8569
7501/8569
7502/8569
7503/8569
7504/8569
7505/8569
7506/8569
7507/8569
7508/8569
7509/8569
7510/8569
7511/8569
7512/8569
7513/8569
7514/8569
7515/8569
7516/8569
7517/8569
7518/8569
7519/8569
7520/8569
7521/8569
7522/8569
7523/8569
7524/8569
7525/8569
7526/8569
7527/8569
7528/8569
7529/8569
7530/8569
7531/8569
7532/8569
7533/8569
7534/8569
7535/8569
7536/8569
7537/8569
7538/8569
7539/8569
7540/8569
7541/8569
7542/8569
7543/8569
7544/8569
7545/8569
7546/8569
7547/8569
7548/8569
7549/8569
7550/8569
7551/8569
7552/8569
7553/8569
7554/8569
7555/8569
7556/8569
7557/8569
7558/8569
7559/8569
7560/8569
7561/8569
7562/8569
7563/8569
7564/8569
7565/8569
7566/8569
7567/8569
7568/8569
7569/8569
7570/8569
7571/8569
7572/8569
7573/8569
7574/8569
7575/8569
7576/8569
7577/8569
7578/8569
7579/8569
7580/8569
7581/8569
7582/8569
7583/8569
7584/8569
7585/8569
7586/8569
7587/8569
7588/8569
7589/8569
7590/8569
7591/8569
7592/8569
7593/8569


8315/8569
8316/8569
8317/8569
8318/8569
8319/8569
8320/8569
8321/8569
8322/8569
8323/8569
8324/8569
8325/8569
8326/8569
8327/8569
8328/8569
8329/8569
8330/8569
8331/8569
8332/8569
8333/8569
8334/8569
8335/8569
8336/8569
8337/8569
8338/8569
8339/8569
8340/8569
8341/8569
8342/8569
8343/8569
8344/8569
8345/8569
8346/8569
8347/8569
8348/8569
8349/8569
8350/8569
8351/8569
8352/8569
8353/8569
8354/8569
8355/8569
8356/8569
8357/8569
8358/8569
8359/8569
8360/8569
8361/8569
8362/8569
8363/8569
8364/8569
8365/8569
8366/8569
8367/8569
8368/8569
8369/8569
8370/8569
8371/8569
8372/8569
8373/8569
8374/8569
8375/8569
8376/8569
8377/8569
8378/8569
8379/8569
8380/8569
8381/8569
8382/8569
8383/8569
8384/8569
8385/8569
8386/8569
8387/8569
8388/8569
8389/8569
8390/8569
8391/8569
8392/8569
8393/8569
8394/8569
8395/8569
8396/8569
8397/8569
8398/8569
8399/8569
8400/8569
8401/8569
8402/8569
8403/8569
8404/8569
8405/8569
8406/8569
8407/8569
8408/8569
8409/8569
8410/8569
8411/8569
8412/8569
8413/8569
8414/8569


In [218]:
new_x = np.stack(x, axis=2)

## Pytorch Machine Learning Shit

In [222]:
import math as m
import torch
from torch.nn import Linear, ReLU, Conv1d, Conv2d, Flatten, Sequential, CrossEntropyLoss, MSELoss, MaxPool1d, MaxPool2d, Dropout, BatchNorm1d, BatchNorm2d

In [223]:
from torch.optim import Adam
from torch import nn
import torchvision

In [227]:
from functools import partial
from dataclasses import dataclass
from collections import OrderedDict

In [236]:
class Conv2dAuto(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.padding = (self.kernel_size[0] // 2, self.kernel_size[1] // 2)
        
conv3x3 = partial(Conv2dAuto, kernel_size=3, bias=False)      

In [237]:
conv = conv3x3(in_channels=32, out_channels=64)
print(conv)

Conv2dAuto(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


In [238]:
def activation_func(activation):
    return nn.ModuleDict([
        ['relu', nn.ReLU(inplace=True)],
        ['leaky_relu', nn.LeakyReLU(negative_slope=0.01, inplace=True)],
        ['selu', nn.SELU(inplace=True)],
        ['none', nn.Identity()]
    ])[activation]

In [253]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, activation='relu'):
        super().__init__()
        self.in_channels, self.out_channels, self.activation = in_channels, out_channels, activation
        self.blocks = nn.Identity()
        self.activate = activation_func(activation)
        self.shortcut = nn.Identity()
        
    def forward(self, x):
        residual = x
        if self.should_apply_shortcut: residual = self.shortcut(x)
        x = self.blocks(x)
        x += residual
        x = self.activate(x)
        
        return x
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.out_channels
    

In [255]:
class ResNetResidualBlock(ResidualBlock):
    def __init__(self, in_channels, out_channels, expansion=1, downsampling=1, conv=conv3x3, *args, **kwargs):
        super().__init__(in_channels, out_channels, *args, **kwargs)
        self.expansion, self.downsampling, self.conv = expansion, downsampling, conv
        self.shortcut = nn.Sequential(
            nn.Conv2d(self.in_channels, self.expanded_channels, kernel_size=1, stride=self.downsampling, bias=False),
            nn.BatchNorm2d(self.expanded_channels)) if self.should_apply_shortcut else None
    
    @property
    def expanded_channels(self):
        return self.out_channels * self.expansion
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.expanded_channels
        

In [261]:
ResNetResidualBlock(32, 64)

ResNetResidualBlock(
  (blocks): Identity()
  (activate): ReLU(inplace=True)
  (shortcut): Sequential(
    (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [262]:
def conv_bn(in_channels, out_channels, conv, *args, **kwargs):
    return nn.Sequential(conv(in_channels, out_channels, *args, **kwargs),
                        nn.BatchNorm2d(out_channels))

In [265]:
class ResNetBasicBlock(ResNetResidualBlock):
    expansion = 1
    def __init__(self, in_channels, out_channels, *args, **kwargs):
        super().__init__(in_channels, out_channels, *args, **kwargs)
        self.blocks = nn.Sequential(
            conv_bn(self.in_channels, self.out_channels, conv=self.conv, bias=False, stride=self.downsampling),
            activation_func(self.activation),
            conv_bn(self.out_channels, self.expanded_channels, conv=self.conv, bias=False),)

In [266]:
dummy = torch.ones((1,32, 10, 10))

block = ResNetBasicBlock(32, 64)
block(dummy).shape

torch.Size([1, 64, 10, 10])

In [269]:
class ResNetBottleNeckBlock(ResNetResidualBlock):
    expansion = 4
    def __init__(self, in_channels, out_channels, *args, **kwargs):
        super().__init__(in_channels, out_channels, expansion=4, *args, **kwargs)
        self.blocks = nn.Sequential(
            conv_bn(self.in_channels, self.out_channels, self.conv, kernel_size=1),
            activation_func(self.activation),
            conv_bn(self.out_channels, self.out_channels, self.conv, kernel_size=3, stride=self.downsampling),
            activation_func(self.activation),
            conv_bn(self.out_channels, self.expanded_channels, self.conv, kernel_size=1),)

In [272]:
dummy = torch.ones((1,32, 10, 10))

block = ResNetBottleNeckBlock(32, 64)
block(dummy).shape

torch.Size([1, 256, 10, 10])

In [274]:
class ResNetLayer(nn.Module):
    def __init__(self, in_channels, out_channels, block=ResNetBottleNeckBlock, n=1, *args, **kwargs):
        super().__init__()
        downsampling = 2 if in_channels != out_channels else 1
        self.blocks = nn.Sequential(
            block(in_channels, out_channels, *args, **kwargs, downsampling=downsampling),
            *[block(out_channels * block.expansion,
                   out_channels, downsampling=1, *args, **kwargs) for _ in range(n-1)])
        
    def forward(self, x):
        x = self.blocks(x)
        return x

In [275]:
dummy = torch.ones((1,64, 48,48))

layer = ResNetLayer(64, 128, block=ResNetBottleNeckBlock, n=3)
layer(dummy).shape

torch.Size([1, 512, 24, 24])

In [276]:
print(layer(dummy))

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         ...,

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 